# Introduction

The purpose of this .ipynb is to generate any-combination of mutation perturbations of motifs across a given set of regions and to plot the predictions alongside the contribution scores. The steps involved in this process are as follows:

1. Import enhancer regions based on annotated bed file.
2. Import motif regions based on curated set.
3. Match (1) and (2) for general overlaps to cut out motifs for filtering.
4. Redo 3, but match accurately and assign `example_idx` to motifs and enhancers for perturbation analysis.
5. Perturb the actual region sequences based on these motif locations in a combinatorial fashion.
6. Plot the predictions and contribution scores
    + Annotate motif locations in addition to which motifs are being removed.

We will be using `ZDTBCG` model with enhancer regions curated by Kaelan based on annotated enhancers. 

# Computational setup

In [1]:
import warnings
warnings.filterwarnings("ignore")
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

#Packages
import os
import sys
import pandas as pd
import numpy as np
from tqdm import tqdm
from pybedtools import BedTool
from bpnet.cli.contrib import bpnet_contrib
from bpnet.preproc import resize_interval

# Settings
os.chdir('/l/Zeitlinger/ZeitlingerLab/Manuscripts/Zelda_and_Nucleosomes/Analysis/analysis/')
pd.set_option('display.max_columns', 100)

# Custom commands
sys.path.insert(0, f'scripts/py')
from bpnet_data_format_functions import myround, myfloor, myceiling, df_to_intervals, tidy_bpnet_predictions_nexus, tidy_bpnet_contributions
from bpnet_motif_functions import remove_palindromic_motif_duplicates, filter_overlapping_motifs_by_priority
from bpnet_perturb_functions import random_seq_onehot, generate_perturbs_across_window, plot_perturbs_across_window

#Pre-existing variables
figure_filepath = 'figures/5b_binding_enhancer_perturbs'
fasta_file = f'../data/indexes/bowtie2/dm6.fa'
model_dir = f'bpnet/models/optimized_model/fold1/'
contrib_file = f'bpnet/preds/fold1/contrib.h5'
modisco_dir = f'bpnet/modisco/fold1'
curated_motifs = f'bed/mapped_motifs/all_instances_curated_0based.bed'
curated_regions = f'bed/mapped_motifs/all_grouped_regions_0based.bed'
enhancer_regions_file = f'bed/enhancers/enhancers_for_models.bed'
tasks = ['Zld', 'Dl', 'Twi', 'Bcd', 'Cad', 'GAF']
enhancer_contrib_file = f'bpnet/preds/fold1/enhancers_for_models.h5'
enhancer_bed_file = f'bed/enhancers/enhancers_for_models.bed3'

task_color_dict = {'Zld': '#d53e4f', 'Dl': '#fc8d59', 'Twi': '#fee08b', 'Bcd': '#e6f598', 'Cad': '#99d594',  'GAF': '#3288bd'}

Using TensorFlow backend.
2022-08-04 10:14:26,999 [INFO] Note: NumExpr detected 64 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2022-08-04 10:14:27,003 [INFO] NumExpr defaulting to 8 threads.


In [2]:
!mkdir -p {figure_filepath}
!mkdir -p tsv/perturbs/binding/enhancer

# Generate contributions across grouped regions

We will generate contribution scores across our set of enhancer regions. This will control for indexes.

In [3]:
#%%script false --no-raise-error
!cat {enhancer_regions_file} | cut -f 1,2,3 > {enhancer_bed_file}

In [4]:
#%%script false --no-raise-error
bpnet_contrib(model_dir = model_dir, 
              output_file = enhancer_contrib_file, 
              method = 'deeplift', fasta_file = fasta_file,
              regions = enhancer_bed_file, 
              overwrite = True)

2022-08-04 10:14:31,107 [INFO] Loading the config files
2022-08-04 10:14:31,112 [INFO] Creating the dataset
TF-MoDISco is using the TensorFlow backend.
2022-08-04 10:14:54,390 [INFO] Using the following interpretation targets:


Bcd/profile/wn
Bcd/counts/pre-act
Cad/profile/wn
Cad/counts/pre-act
Dl/profile/wn
Dl/counts/pre-act
GAF/profile/wn
GAF/counts/pre-act
Twi/profile/wn
Twi/counts/pre-act
Zld/profile/wn
Zld/counts/pre-act


  0%|          | 0/1.0 [00:00<?, ?it/s]

DeepExplain: running "deeplift" explanation method (5)
Model with multiple inputs:  True
DeepExplain: running "deeplift" explanation method (5)
Model with multiple inputs:  True
DeepExplain: running "deeplift" explanation method (5)
Model with multiple inputs:  True
DeepExplain: running "deeplift" explanation method (5)
Model with multiple inputs:  True
DeepExplain: running "deeplift" explanation method (5)
Model with multiple inputs:  True
DeepExplain: running "deeplift" explanation method (5)
Model with multiple inputs:  True
DeepExplain: running "deeplift" explanation method (5)
Model with multiple inputs:  True
DeepExplain: running "deeplift" explanation method (5)
Model with multiple inputs:  True
DeepExplain: running "deeplift" explanation method (5)
Model with multiple inputs:  True
DeepExplain: running "deeplift" explanation method (5)
Model with multiple inputs:  True
DeepExplain: running "deeplift" explanation method (5)
Model with multiple inputs:  True
DeepExplain: running 

100%|██████████| 1/1.0 [02:15<00:00, 135.98s/it]
2022-08-04 10:17:12,254 [INFO] Done. Contribution score file was saved to: bpnet/preds/fold1/enhancers_for_models.h5


# Connect curated motifs with enhancer ids 

The enhancer contribution file will have each enhancer as an `example_idx`. Match the curated motifs to the correct range such that we can generate perturbations.

In [5]:
#Find overlaps between motifs to cut down on motifs
motif_ov_df = BedTool(curated_motifs).intersect(BedTool(enhancer_bed_file)).to_dataframe()

In [6]:
#Import motifs
motifs_df = BedTool(curated_motifs).to_dataframe()
motifs_df.columns = ['example_chrom','start','end','name','score','strand']

#Filter motifs by their overlap with enhancers
motifs_df = motifs_df[motifs_df.name.isin(motif_ov_df.name)]

#Separate motif name
motifs_df['pattern_len'] = motifs_df['end'] - motifs_df['start']
motifs_df['pattern_name'] = [n.split('_')[0] for n in motifs_df.name]
motifs_df['motif_id'] = [n.split('_')[1] for n in motifs_df.name]
motifs_df['region_id'] = [n.split('_')[2] for n in motifs_df.name]
motifs_df.shape

(302, 10)

Get enhancer coordinates over the exact contribution window.

In [7]:
from bpnet.cli.contrib import ContribFile

#Extract enhancer coordinates
contrib_file = ContribFile(enhancer_contrib_file)
enhancer_coords_df = contrib_file.get_ranges()

In [8]:
enhancer_coords_df.head()

,chrom,start,end,strand,interval_from_task,idx
0,chr3R,8698828,8699828,.,,0
1,chr3R,8700693,8701693,.,,1
2,chr3R,8694217,8695217,.,,2
3,chr2R,25236999,25237999,.,,3
4,chr2R,25225884,25226884,.,,4


Check for which motif overlaps with which enhancer. Filter out motifs with no overlap.

In [9]:
motifs_with_eidx = pd.DataFrame()
for i,motif in tqdm(motifs_df.iterrows()):
    enhancer_match = []
    for j,enh in enhancer_coords_df.iterrows():
        if motif.example_chrom == enh.chrom and motif.start > enh.start and motif.end < enh.end:
            row_new = motif
            row_new['example_idx'] = enh.idx
            row_new['pattern_start'] = motif.start - enh.start
            row_new['pattern_end'] = motif.end - enh.start
            row_new['pattern'] = motif.pattern_name
            motifs_with_eidx = motifs_with_eidx.append(row_new)

302it [00:10, 29.53it/s]


In [10]:
motifs_with_eidx.head()

,end,example_chrom,example_idx,motif_id,name,pattern,pattern_end,pattern_len,pattern_name,pattern_start,region_id,score,start,strand
1330,2454716.0,chr2L,20.0,64675,Zld_64675_810,Zld,698.0,7.0,Zld,691.0,810,0.0,2454709.0,-
1331,2456357.0,chr2L,19.0,64676,Zld_64676_811,Zld,130.0,7.0,Zld,123.0,811,0.0,2456350.0,-
1332,2456498.0,chr2L,19.0,31102,Dl_31102_811,Dl,271.0,11.0,Dl,260.0,811,0.0,2456487.0,-
1333,2456534.0,chr2L,19.0,64677,Zld_64677_811,Zld,307.0,7.0,Zld,300.0,811,0.0,2456527.0,-
1334,2456621.0,chr2L,19.0,31103,Dl_31103_811,Dl,394.0,11.0,Dl,383.0,811,0.0,2456610.0,+


Show how many regions have motifs mapped across them.

In [11]:
motifs_with_eidx.example_idx.value_counts().value_counts()

9     6
7     6
8     5
6     5
12    4
5     4
4     4
3     4
10    3
2     2
1     2
13    1
Name: example_idx, dtype: int64

In [12]:
motifs_with_eidx.sort_values(['example_idx']).head(n = 10)

,end,example_chrom,example_idx,motif_id,name,pattern,pattern_end,pattern_len,pattern_name,pattern_start,region_id,score,start,strand
43661,8699205.0,chr3R,0.0,13978,Bcd_13978_27777,Bcd,377.0,7.0,Bcd,370.0,27777,0.0,8699198.0,-
43667,8699425.0,chr3R,0.0,13983,Bcd_13983_27777,Bcd,597.0,7.0,Bcd,590.0,27777,0.0,8699418.0,+
43666,8699413.0,chr3R,0.0,13982,Bcd_13982_27777,Bcd,585.0,7.0,Bcd,578.0,27777,0.0,8699406.0,-
43665,8699378.0,chr3R,0.0,13981,Bcd_13981_27777,Bcd,550.0,7.0,Bcd,543.0,27777,0.0,8699371.0,+
43664,8699288.0,chr3R,0.0,13980,Bcd_13980_27777,Bcd,460.0,7.0,Bcd,453.0,27777,0.0,8699281.0,+
43662,8699245.0,chr3R,0.0,13979,Bcd_13979_27777,Bcd,417.0,7.0,Bcd,410.0,27777,0.0,8699238.0,-
43660,8699157.0,chr3R,0.0,68349,Zld_68349_27777,Zld,329.0,7.0,Zld,322.0,27777,0.0,8699150.0,+
43663,8699263.0,chr3R,0.0,68350,Zld_68350_27777,Zld,435.0,7.0,Zld,428.0,27777,0.0,8699256.0,-
43672,8701461.0,chr3R,1.0,28642,Cad_28642_27779,Cad,768.0,10.0,Cad,758.0,27779,0.0,8701451.0,+
43668,8701021.0,chr3R,1.0,68351,Zld_68351_27778,Zld,328.0,7.0,Zld,321.0,27778,0.0,8701014.0,+


# Generate perturbation plots

Here we will undergo the following steps to generate perturbation plots:

1. Import the formatted CWM-scanned motifs from above.
2. For each enhancer:
    + Perturb the motifs from (1) in a combinatorial fashion.
    + Generate (1) profile predictions, (2) contribution scores, and (3) a list of the mutations generated.
    + Tidy information and save as .tsv.gz for more efficient plotting in R.

In [13]:
#Reassign motifs for clarity
dfi = motifs_with_eidx
dfi['pattern_name_unique'] = dfi['pattern_name'] + '-' + (dfi.groupby(['example_idx','pattern_name']).cumcount()+1).astype(str)
dfi[['start','end','example_idx','pattern_start',
     'pattern_end','pattern_len']] = dfi[['start','end','example_idx','pattern_start',
                                          'pattern_end','pattern_len']].astype(int)

#Reassign enhancers for clarity
enhancer_coords_df['name'] = BedTool(enhancer_regions_file).to_dataframe().name
enhancers_df = enhancer_coords_df

Iterate through enhancers and then collect perturbation profiles for each single motif mutation across the enhancers.

In [14]:
max_perturbs_per_mut = 1
xrange = [100, 900]

for ei in tqdm(dfi.example_idx.unique()):
    #Subset motifs by window
    dfi_across_window = dfi[dfi.example_idx==ei]
    enhancer_df = enhancers_df[enhancers_df.idx==ei]
    
    #Generate perturbation predictions, contribution, and a mutant list
    perturb_preds, _, muts = generate_perturbs_across_window(dfi = dfi_across_window, 
                                                             contrib_file = enhancer_contrib_file, 
                                                             comb_max = 1,
                                                             model_dir = model_dir, 
                                                             return_contrib = False)
    
    #Save profile predictions
    perturb_preds_format_dict = {m: {k: v[i] for k,v in perturb_preds.items()} for i,m in enumerate(muts)}
    profile_preds_df = pd.DataFrame()
    for i,m in enumerate(muts):
        profile_pred_df = tidy_bpnet_predictions_nexus(perturb_preds_format_dict[m], tasks = perturb_preds.keys())
        profile_pred_df['mut']=m
        profile_preds_df = profile_preds_df.append(profile_pred_df)
    profile_preds_df['enhancer_name'] = enhancer_df.name
    profile_preds_df['enhancer_idx'] = enhancer_df.idx
    profile_preds_df['genomic_position_0based'] = int(enhancer_df['start']) + profile_preds_df.position
    profile_preds_df
    profile_preds_df.to_csv(f"tsv/perturbs/binding/enhancer/{enhancer_df['name'].iloc[0]}_predictions.tsv.gz", 
                            index = False)

100%|██████████| 46/46 [1:01:09<00:00, 79.77s/it] 


Extract enhancer information for subsequent plotting in later analysis. 

In [15]:
max_perturbs_per_mut = 1
xrange = [100, 900]

for ei in tqdm(dfi.example_idx.unique()):
    #Subset motifs by window
    dfi_across_window = dfi[dfi.example_idx==ei]
    enhancer_df = enhancers_df[enhancers_df.idx==ei]
    
    #Save motif information
    dfi_across_window['enhancer_name'] = enhancer_df.name.iloc[0]
    dfi_across_window['enhancer_idx'] = enhancer_df.idx.iloc[0]
    dfi_across_window['enhancer_start_0based'] = enhancer_df.start.iloc[0]
    dfi_across_window['enhancer_end_0based'] = enhancer_df.end.iloc[0]
    dfi_across_window.to_csv(f"tsv/perturbs/binding/enhancer/{enhancer_df['name'].iloc[0]}_motif_info_0based.csv.gz", 
                             index = False)

100%|██████████| 46/46 [00:00<00:00, 50.61it/s]


In [16]:
dfi_across_window

,end,example_chrom,example_idx,motif_id,name,pattern,pattern_end,pattern_len,pattern_name,pattern_start,region_id,score,start,strand,pattern_name_unique,enhancer_name,enhancer_idx,enhancer_start_0based,enhancer_end_0based
65320,15646675,chrX,45,29365,Cad_29365_41267,Cad,117,10,Cad,107,41267,0.0,15646665,+,Cad-1,"sog_shadow,type3",45,15646558,15647558
65321,15646765,chrX,45,68698,Zld_68698_41267,Zld,207,7,Zld,200,41267,0.0,15646758,+,Zld-1,"sog_shadow,type3",45,15646558,15647558
65322,15646786,chrX,45,32560,Dl_32560_41267,Dl,228,11,Dl,217,41267,0.0,15646775,+,Dl-1,"sog_shadow,type3",45,15646558,15647558
65323,15646839,chrX,45,60758,Twi_60758_41267,Twi,281,9,Twi,272,41267,0.0,15646830,-,Twi-1,"sog_shadow,type3",45,15646558,15647558
65324,15646851,chrX,45,29366,Cad_29366_41267,Cad,293,10,Cad,283,41267,0.0,15646841,-,Cad-2,"sog_shadow,type3",45,15646558,15647558
65325,15646959,chrX,45,68699,Zld_68699_41267,Zld,401,7,Zld,394,41267,0.0,15646952,-,Zld-2,"sog_shadow,type3",45,15646558,15647558
65326,15647010,chrX,45,29367,Cad_29367_41267,Cad,452,10,Cad,442,41267,0.0,15647000,-,Cad-3,"sog_shadow,type3",45,15646558,15647558
65327,15647136,chrX,45,68700,Zld_68700_41267,Zld,578,7,Zld,571,41267,0.0,15647129,-,Zld-3,"sog_shadow,type3",45,15646558,15647558
65328,15647151,chrX,45,32561,Dl_32561_41267,Dl,593,11,Dl,582,41267,0.0,15647140,-,Dl-2,"sog_shadow,type3",45,15646558,15647558
65329,15647172,chrX,45,60759,Twi_60759_41267,Twi,614,9,Twi,605,41267,0.0,15647163,+,Twi-2,"sog_shadow,type3",45,15646558,15647558
